## Twitter hates speech detection using FastText

In [1]:
import re
import string
import nltk
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier
from nltk.tokenize import word_tokenize

from wordcloud import WordCloud,STOPWORDS
import matplotlib.pyplot as plt
%matplotlib inline

from subprocess import check_output
#nltk.download('stopwords')
from nltk.corpus import stopwords
#from nltk.stem.porter import PorterStemme
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [2]:
import pandas as pd

df= pd.read_csv("TwitterHate.csv")
print(df.shape)
df.head(3)

(31962, 3)


,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty


**Drop NA values**

In [3]:
df.dropna(inplace=True)
df.shape

(31962, 3)

In [4]:
df.label.unique()

array([0, 1], dtype=int64)

In [5]:
df['label_map'] = df['label'].map({0:"none_hate",1:"hate"})

In [6]:
df.head()

,id,label,tweet,label_map
0,1,0,@user when a father is dysfunctional and is s...,none_hate
1,2,0,@user @user thanks for #lyft credit i can't us...,none_hate
2,3,0,bihday your majesty,none_hate
3,4,0,#model i love u take with u all the time in ...,none_hate
4,5,0,factsguide: society now #motivation,none_hate


In [7]:
df.label_map.unique()

array(['none_hate', 'hate'], dtype=object)

In [8]:
df.label_map.value_counts()

none_hate    29720
hate          2242
Name: label_map, dtype: int64

In [9]:
df.tweet[1]

"@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx.    #disapointed #getthanked"

In [10]:
def preprocessing(data):
    stemmer = nltk.stem.RSLPStemmer()
    all_stopwords = stopwords.words('english')
    all_stopwords.remove('not')
    corpus = []
    for tweet in data:
                
        review = re.sub(r"@[A-Za-z0-9_]+", " ", tweet)
        review = re.sub('RT', ' ', review)
        review = re.sub(r"https?://[A-Za-z0-9./]+", " ", review)
        review = re.sub(r"https?", " ", review)
        review = re.sub('[^a-zA-Z]', ' ', review)
        review = review.lower()
        review = review.split()
        ps = PorterStemmer()
        review = [ps.stem(word) for word in review if not word in set(all_stopwords) if len(word) > 2]
        review = ' '.join(review)
        corpus.append(review)

    return np.array(corpus)

In [11]:
df['cleen_tweet'] = preprocessing(df['tweet'].values)

In [12]:

df.head()

,id,label,tweet,label_map,cleen_tweet
0,1,0,@user when a father is dysfunctional and is s...,none_hate,father dysfunct selfish drag kid dysfunct run
1,2,0,@user @user thanks for #lyft credit i can't us...,none_hate,thank lyft credit use caus offer wheelchair va...
2,3,0,bihday your majesty,none_hate,bihday majesti
3,4,0,#model i love u take with u all the time in ...,none_hate,model love take time
4,5,0,factsguide: society now #motivation,none_hate,factsguid societi motiv


#### Train a fasttext model, it expects labels to be specified with label prefix. I have just create a third column in the dataframe that has label as well as the Tweets

In [13]:
df['_label_map'] = '__label__' + df['label_map'].astype(str)
df.head(5)

,id,label,tweet,label_map,cleen_tweet,_label_map
0,1,0,@user when a father is dysfunctional and is s...,none_hate,father dysfunct selfish drag kid dysfunct run,__label__none_hate
1,2,0,@user @user thanks for #lyft credit i can't us...,none_hate,thank lyft credit use caus offer wheelchair va...,__label__none_hate
2,3,0,bihday your majesty,none_hate,bihday majesti,__label__none_hate
3,4,0,#model i love u take with u all the time in ...,none_hate,model love take time,__label__none_hate
4,5,0,factsguide: society now #motivation,none_hate,factsguid societi motiv,__label__none_hate


In [14]:
df['Label_Tweet'] = df['_label_map'] + ' ' + df['cleen_tweet']
df.head()

,id,label,tweet,label_map,cleen_tweet,_label_map,Label_Tweet
0,1,0,@user when a father is dysfunctional and is s...,none_hate,father dysfunct selfish drag kid dysfunct run,__label__none_hate,__label__none_hate father dysfunct selfish dra...
1,2,0,@user @user thanks for #lyft credit i can't us...,none_hate,thank lyft credit use caus offer wheelchair va...,__label__none_hate,__label__none_hate thank lyft credit use caus ...
2,3,0,bihday your majesty,none_hate,bihday majesti,__label__none_hate,__label__none_hate bihday majesti
3,4,0,#model i love u take with u all the time in ...,none_hate,model love take time,__label__none_hate,__label__none_hate model love take time
4,5,0,factsguide: society now #motivation,none_hate,factsguid societi motiv,__label__none_hate,__label__none_hate factsguid societi motiv


In [15]:
df.Label_Tweet[1]

'__label__none_hate thank lyft credit use caus offer wheelchair van pdx disapoint getthank'

**Train Test Split**

In [16]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2)

In [17]:
train.shape, test.shape

((25569, 7), (6393, 7))

In [18]:
train.to_csv("Fasttext_Tweet.train", columns=["Label_Tweet"], index=False, header=False)
test.to_csv("Fasttext_Tweet.test", columns=["Label_Tweet"], index=False, header=False)

**Train the model and evaluate performance**

In [19]:
import fasttext

model = fasttext.train_supervised(input="Fasttext_Tweet.train",lr=0.8, epoch=125, wordNgrams=2)
model.test("Fasttext_Tweet.test")

(6393, 0.9630846238072892, 0.9630846238072892)

First parameter (6393) is test size. Second and third parameters are precision and recall respectively. You can see we are getting around 96% precision which is pretty good

**Now let's do prediction for few some tweets**

In [20]:
model.predict("i dont like when women got lesser sallary than man")

(('__label__none_hate',), array([0.89875108]))

In [21]:
model.predict("Sometimes, you think that you want to disappear, but all you really want is to be found.")

(('__label__none_hate',), array([0.98239154]))

In [22]:
model.predict("Even the darkest night will end, and the sun will rise.")

(('__label__none_hate',), array([0.99907213]))

In [23]:
model.predict("You are not born a winner. You are not born a loser. You are born a chooser.")

(('__label__none_hate',), array([0.99944967]))

In [24]:
model.predict("	tweet78	@user hey, white people: you can call people 'white' by @user  #race  #identity #medâ¦")

(('__label__hate',), array([0.99872649]))

In [25]:
model.predict("	tweet57	@user lets fight against ")

(('__label__none_hate',), array([0.65334427]))